In [102]:
import os
import fnmatch
import pandas as pd
import numpy as np
from datetime import datetime
import shutil
import sys
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

PROD_CONCAT = "ProdConcat"
PRECIO_VENTA = "Precio Venta"
CANTIDAD = "Cantidad"
FECHA = "Fecha"
ALMACEN = "Almacen"
IVA = .16
CIEN = 100
MES = "Mes"
PREDICCION = "Prediccion Diciembre"

CLASIFICACION = "CLASIFICACION"
N1 = "N1"
N2 = "N2"
N3 = "N3"

def unir_dataframes(lista_df):
    """
    Une una lista de DataFrames en uno solo, validando que todos tengan las mismas columnas.

    :param lista_df: Lista de DataFrames a unir.
    :return: DataFrame combinado si todos los DataFrames tienen las mismas columnas.
    :raises ValueError: Si los DataFrames no tienen las mismas columnas.
    """
    # Validar que la lista no esté vacía
    if not lista_df:
        raise ValueError("La lista de DataFrames está vacía.")
    
    # Obtener las columnas del primer DataFrame como referencia
    columnas_referencia = lista_df[0].columns
    
    # Verificar que todos los DataFrames tengan las mismas columnas
    for i, df in enumerate(lista_df):
        if not df.columns.equals(columnas_referencia):
            raise ValueError(f"El DataFrame en la posición {i} no tiene las mismas columnas.")
    
    # Concatenar los DataFrames si pasan la validación
    df_resultado = pd.concat(lista_df, ignore_index=True)
    return df_resultado

def generar_excel_by_dataframe(df, nombre_base):
    """
    Genera un archivo Excel a partir de un DataFrame, añadiendo la fecha y hora actual al nombre del archivo.
    :param df: DataFrame a exportar.
    :param nombre_base: Nombre base del archivo (sin extensión).
    :return: Ruta completa del archivo generado.
    """
    try:
        # Obtener la fecha y hora actuales en formato 'YYYYMMDD_HHMMSS'
        fecha_hora = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
        
        # Construir el nombre del archivo
        nombre_archivo = f"{nombre_base}_{fecha_hora}.xlsx"
        
        # Exportar el DataFrame a Excel
        df.to_excel(nombre_archivo, index=False, engine="openpyxl")
        
        print(f"Archivo Excel generado exitosamente: {nombre_archivo}")
        return nombre_archivo
    except Exception as e:
        print(f"Error al generar el archivo Excel: {e}")
        return None

def archivos_excel_by_coincidencia(directorio: str, cadena: str):
    archivos_excel = []
    patron = f"*{cadena}*.xlsx"
    
    for archivo in os.listdir(directorio):
        if fnmatch.fnmatch(archivo, patron):
            archivos_excel.append(archivo)

    return archivos_excel
    
def filtrar_columnas_dataframe(dataframe, columnas):
    """
    Devuelve un DataFrame que contiene solo las columnas especificadas.

    :param dataframe: DataFrame de pandas del que se desea conservar las columnas.
    :param columnas: Lista de nombres de columnas a conservar.
    :return: DataFrame con solo las columnas especificadas.
    """
    try:
        # Verificar cuáles columnas existen en el DataFrame
        columnas_existentes = [col for col in columnas if col in dataframe.columns]
        columnas_no_existentes = [col for col in columnas if col not in dataframe.columns]

        if columnas_no_existentes:
            print(f"Las siguientes columnas no existen en el DataFrame: {columnas_no_existentes}")

        # Columnas filtradas
        dataframe = dataframe[columnas_existentes]
    except Exception as e:
        print(f"Se produjo un error al intentar filtrar las columnas: {e}")

    return dataframe
    
def validar_datos_numericos_dataframe(dataframe, columnas):
    """
    Reemplaza ceros en las columnas especificadas de un DataFrame con NaN.
    :param dataframe: DataFrame en el que se procesarán las columnas.
    :param columnas: Lista de nombres de columnas donde se reemplazarán los ceros por NaN.
    :return: DataFrame con los ceros reemplazados por NaN en las columnas especificadas.
    """
    try:
        # Validar que las columnas existan en el DataFrame
        columnas_validas = [col for col in columnas if col in dataframe.columns]
        # Reemplazar ceros por NaN en las columnas válidas
        dataframe[columnas_validas] = dataframe[columnas_validas].replace(0, np.nan)
        print(f"Ceros reemplazados por NaN en las columnas: {columnas_validas}")
    except Exception as e:
        print(f"Error al reemplazar ceros por NaN: {e}")
    return dataframe
    
def crear_carpeta(base_nombre_carpeta, ruta_base="."):
    """
    Crea una carpeta con un nombre que incluye la fecha y hora actual al final.
    
    :param base_nombre_carpeta: Nombre base para la carpeta.
    :param ruta_base: Ruta donde se creará la carpeta. Por defecto, en el directorio actual.
    :return: Ruta completa de la carpeta creada.
    """
    try:
        # Obtener la fecha y hora actuales en formato 'YYYY-MM-DDTHH-MM-SS'
        fecha_hora = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
        
        # Construir el nombre completo de la carpeta
        nombre_completo_carpeta = f"{base_nombre_carpeta}_{fecha_hora}"
        ruta_completa_carpeta = os.path.join(ruta_base, nombre_completo_carpeta)
        
        # Crear la carpeta
        os.makedirs(ruta_completa_carpeta, exist_ok=True)
        print(f"Carpeta creada: {ruta_completa_carpeta}")
        return ruta_completa_carpeta
    except Exception as e:
        print(f"Error al crear la carpeta: {e}")
        return None

def mover_archivos_a_carpeta(lista_archivos, carpeta_destino):
    """
    Mueve una lista de archivos a una carpeta destino.
    :param lista_archivos: Lista con las rutas de los archivos a mover.
    :param carpeta_destino: Ruta de la carpeta destino.
    :return: None
    """
    try:
        # Crear la carpeta destino si no existe
        carpeta_destino = crear_carpeta(carpeta_destino)
        for archivo in lista_archivos:
            if os.path.isfile(archivo):  # Verificar que el archivo existe
                destino = os.path.join(carpeta_destino, os.path.basename(archivo))  # Ruta destino
                shutil.move(archivo, destino)  # Mover el archivo
                print(f"Archivo movido: {archivo} -> {destino}")
            else:
                print(f"El archivo no existe: {archivo}")
    except Exception as e:
        print(f"Error al mover archivos: {e}")

def validar_archivos(lista_archivos):
    """
    Valida que una lista de archivos no contenga valores vacíos.
    :param lista_archivos: Lista de rutas de archivos.
    :return: True si todos los archivos son válidos, False si hay archivos faltantes.
    """
    # Filtrar archivos vacíos
    archivos_faltantes = [archivo for archivo in lista_archivos if archivo == ""]
    
    if archivos_faltantes:
        print("Error: Faltan archivos necesarios para el proceso del reporte.")
       
    
    print("Todos los archivos son válidos.")
    return True

def validar_listas_archivos_no_vacias(lista_de_listas):
    """
    Valida que ninguna lista dentro de una lista de listas esté vacía.
    
    :param lista_de_listas: Lista de listas de strings.
    :return: True si ninguna lista está vacía.
    :raises ValueError: Si alguna lista está vacía.
    """
    for i, lista in enumerate(lista_de_listas):
        if not lista:  # Verifica si la lista está vacía
            print("Error: Faltan archivos necesarios para el proceso del reporte.")
            sys.exit(1)  # Rompe la ejecución con un código de error
            return False
    
    return True

def crear_dataframe_from_excel(archivo: str, columnas: list = None, hoja: str = None):
    """
    Lee un archivo de Excel y crea un DataFrame filtrado con las columnas especificadas.

    :param archivo: Ruta del archivo Excel a leer.
    :param columnas: Lista de nombres de columnas que se desean extraer del archivo.
    :param hoja: Nombre de la hoja a leer. Si no se especifica, se lee la primera hoja por defecto.
    :return: DataFrame con las columnas filtradas, o None si ocurre un error.
    """
    try:
        # Inicializar el DataFrame
        df = None
        # Leer el archivo de Excel
        # Si no se especifica una hoja, se lee la primera por defecto
        if hoja is None:
            df = pd.read_excel(archivo, engine='openpyxl')
        else:
            # Si se especifica una hoja, se intenta leer esa hoja en particular
            df = pd.read_excel(archivo, engine='openpyxl', sheet_name=hoja)
        # Filtrar las columnas especificadas por el usuario
        if columnas:
            df = df[columnas]
        # Devolver el DataFrame filtrado
        return df
    # Manejo de excepciones
    except FileNotFoundError:
        # Error si el archivo no se encuentra en la ruta especificada
        print(f"El archivo {archivo} no fue encontrado.")
    except KeyError as e:
        # Error si una o más columnas no existen en el archivo
        print(f"Una o más columnas no se encuentran en el archivo: {e}")
    except ValueError:
        # Error si la hoja especificada no existe en el archivo
        print(f"La hoja '{hoja}' no existe en el archivo {archivo}.")
    except Exception as e:
        # Captura cualquier otro error no previsto
        print(f"Se produjo un error al procesar el archivo: {e}")

def crear_dataframe_unido_por_coincidencia_excel(directorio: str = "./", cadena: str = "", columnas: list = None, hoja: str = None):
    """
    Busca archivos Excel en un directorio que coincidan con una cadena en su nombre, 
    los convierte en DataFrames (opcionalmente filtrando columnas y seleccionando una hoja específica)
    y los une en un solo DataFrame.

    :param directorio: Ruta del directorio donde buscar archivos Excel (por defecto "./").
    :param cadena: Cadena de coincidencia para buscar archivos.
    :param columnas: Lista de columnas a seleccionar de cada archivo (opcional). Si no se proporciona, se cargan todas.
    :param hoja: Nombre de la hoja a leer (opcional). Si no se proporciona, se lee la primera hoja.
    :return: DataFrame unido de todos los archivos encontrados y leídos correctamente.
    :raises FileNotFoundError: Si no se encuentra ningún archivo que coincida con la búsqueda.
    :raises ValueError: Si no se pudieron leer archivos o unir DataFrames correctamente.
    """
    
    # Buscar archivos Excel en el directorio que contengan la cadena en su nombre
    archivos_coincidentes = archivos_excel_by_coincidencia(directorio, cadena)
    
    # Si no se encuentran archivos, lanzar un error
    if not archivos_coincidentes:
        raise FileNotFoundError(f"No se encontraron archivos que coincidan con '{cadena}' en {directorio}.")

    # Lista para almacenar los DataFrames generados a partir de los archivos encontrados
    lista_df = []

    # Iterar sobre cada archivo encontrado
    for archivo in archivos_coincidentes:
        ruta_completa = os.path.join(directorio, archivo)  # Crear la ruta completa del archivo
        
        try:
            # Crear un DataFrame a partir del archivo Excel, con la opción de filtrar columnas y seleccionar hoja
            df = crear_dataframe_from_excel(archivo=ruta_completa, columnas=columnas, hoja=hoja)
            lista_df.append(df)  # Añadir el DataFrame a la lista
        
        except Exception as e:
            # Capturar errores durante la lectura de los archivos
            print(f"Error al leer el archivo {archivo}: {e}")
    
    # Si hay DataFrames válidos en la lista, unirlos
    if lista_df:
        return unir_dataframes(lista_df)
    
    # Si no se pudieron leer los archivos correctamente, lanzar un error
    else:
        raise ValueError("No se pudieron leer los archivos correctamente.")

def ordenar_columnas_al_lado(df, columna_referencia, columnas_a_mover):
    """
    Ordena una lista de columnas al lado de una columna de referencia en un DataFrame.

    :param df: DataFrame a modificar.
    :param columna_referencia: Columna donde se insertarán las demás columnas al lado.
    :param columnas_a_mover: Lista de columnas que se deben mover al lado de la columna de referencia.
    :return: DataFrame con las columnas reordenadas.
    """
    try:
        # Validar que las columnas estén en el DataFrame
        columnas_faltantes = [col for col in columnas_a_mover + [columna_referencia] if col not in df.columns]
        if columnas_faltantes:
            raise ValueError(f"Las siguientes columnas no están en el DataFrame: {columnas_faltantes}")

        # Obtener la lista de columnas actuales
        columnas_actuales = df.columns.tolist()

        # Determinar la posición de la columna de referencia
        indice_referencia = columnas_actuales.index(columna_referencia)

        # Crear una nueva lista de columnas manteniendo el orden original
        nuevas_columnas = []
        for col in columnas_actuales:
            if col == columna_referencia:
                # Insertar la columna de referencia
                nuevas_columnas.append(col)
                # Insertar las columnas a mover justo después de la columna de referencia
                nuevas_columnas.extend([c for c in columnas_a_mover if c in columnas_actuales])
            elif col not in columnas_a_mover:
                # Mantener otras columnas que no se están moviendo
                nuevas_columnas.append(col)

        # Reordenar el DataFrame con las nuevas columnas
        df = df[nuevas_columnas]

        return df

    except Exception as e:
        print(f"Error al reordenar las columnas: {e}")
        return df  # Devolver el DataFrame original en caso de error



In [103]:
def obtener_dataframe_ventas():
    columnas_ventas_acc_tel = [ALMACEN, PROD_CONCAT, CANTIDAD, "PrecioVenta", FECHA]
    df_ventas_acc_tel = crear_dataframe_unido_por_coincidencia_excel(directorio="./Ventas", cadena="Analisis de Ventas", columnas=columnas_ventas_acc_tel)
    df_ventas_acc_tel = validar_datos_numericos_dataframe(df_ventas_acc_tel, [CANTIDAD,"PrecioVenta"])
    # Definir el mapeo de columnas para renombrar
    mapeo_columnas_acc_tel = {
        "PrecioVenta": PRECIO_VENTA
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_ventas_acc_tel.rename(columns=mapeo_columnas_acc_tel, inplace=True)
    df_ventas_acc_tel = validar_datos_numericos_dataframe(df_ventas_acc_tel, [CANTIDAD, PRECIO_VENTA])

    columnas_ventas_refacc = ["Almacén Salida Reparación", "Producto", CANTIDAD, "PrecioVentaSinIva", "Fecha Salida Reparación"]
    df_ventas_refacc = crear_dataframe_unido_por_coincidencia_excel(directorio="./Ventas", cadena="Refacciones_Consumidas", columnas=columnas_ventas_refacc)
    # Definir el mapeo de columnas para renombrar
    mapeo_columnas_refacc = {
        "Almacén Salida Reparación": ALMACEN,
        "Producto": PROD_CONCAT,
        "PrecioVentaSinIva": PRECIO_VENTA,
        "Fecha Salida Reparación": FECHA
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_ventas_refacc.rename(columns=mapeo_columnas_refacc, inplace=True)
    df_ventas_refacc = validar_datos_numericos_dataframe(df_ventas_refacc, [CANTIDAD, PRECIO_VENTA])
    # Calcular el 16% y sumarlo a la columna Costo_Compra
    df_ventas_refacc[PRECIO_VENTA] = df_ventas_refacc[PRECIO_VENTA]+(df_ventas_refacc[PRECIO_VENTA]*IVA)
    #Fusionamos las ventas de accesorios, telefonía y refacciones
    df_ventas_unificadas = unir_dataframes([df_ventas_acc_tel, df_ventas_refacc])
    # Asegurarse de que la columna Fecha está en el formato correcto
    df_ventas_unificadas[FECHA] = pd.to_datetime(df_ventas_unificadas[FECHA], format='%b %d %Y %I:%M%p')
    # Extraer el número del mes de la columna 'Fecha'
    df_ventas_unificadas[MES] = df_ventas_unificadas[FECHA].dt.month
    df_ventas_unificadas.drop(columns=[FECHA], inplace=True)
    return df_ventas_unificadas

In [104]:
def filtrar_ventas_por_mes(df_ventas, lista_meses):
    """
    Filtra el DataFrame de ventas por los meses especificados en la lista.

    :param df_ventas: DataFrame que contiene una columna 'Mes' en formato numerico del 1 al 12
    :param lista_meses: Lista de números que representan los meses a filtrar (1 para enero, 2 para febrero, etc.).
    :return: DataFrame filtrado con las ventas de los meses indicados.
    """
    try:
        # Filtrar el DataFrame según los meses especificados
        df_filtrado = df_ventas[df_ventas['Mes'].isin(lista_meses)]
        return df_filtrado
    except Exception as e:
        print(f"Error al filtrar por mes: {e}")
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error


In [105]:
def agrupa_ventas_by_prodconcat(df_ventas):
    df_ventas = filtrar_columnas_dataframe(df_ventas, [PROD_CONCAT, ALMACEN, CANTIDAD, MES])
    df_ventas = df_ventas.groupby([PROD_CONCAT, ALMACEN, MES]).agg({
        CANTIDAD: 'sum'
    }).reset_index()
    return df_ventas

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

def predecir_ventas_diciembre_rl(df_ventas):
    # Validar que el DataFrame tenga las columnas necesarias
    columnas_requeridas = [ALMACEN, PROD_CONCAT, CANTIDAD, MES]
    for columna in columnas_requeridas:
        if columna not in df_ventas.columns:
            raise ValueError(f"Falta la columna {columna} en el DataFrame.")

    # Crear una copia del DataFrame para evitar modificar el original
    df = df_ventas.copy()

    # Codificar 'Almacen' y 'ProdConcat' como números (Label Encoding)
    df[ALMACEN] = df[ALMACEN].astype('category').cat.codes
    df[PROD_CONCAT] = df[PROD_CONCAT].astype('category').cat.codes

    # Definir características (X) y la variable objetivo (y)
    X = df[[ALMACEN, PROD_CONCAT, MES]]
    y = df[CANTIDAD]

    # Dividir datos en entrenamiento y prueba (80% - 20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # Crear y entrenar el modelo de regresión lineal
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)

    # Hacer predicciones sobre el conjunto de prueba
    y_pred = modelo.predict(X_test)

    # Calcular el error medio absoluto
    error = mean_absolute_error(y_test, y_pred)
    print(f"Error Medio Absoluto (MAE): {error:.2f} unidades.")

    # Predecir ventas para diciembre (Mes = 12)
    diciembre = X.copy()
    diciembre[MES] = 12

    # Hacer predicción para diciembre
    diciembre['Prediccion Diciembre'] = modelo.predict(diciembre)

    # Descodificar 'Almacen' y 'ProdConcat' para devolver los valores originales
    diciembre[ALMACEN] = pd.Categorical.from_codes(diciembre[ALMACEN], df_ventas[ALMACEN].astype('category').cat.categories)
    diciembre[PROD_CONCAT] = pd.Categorical.from_codes(diciembre[PROD_CONCAT], df_ventas[PROD_CONCAT].astype('category').cat.categories)

    # Filtrar y mostrar predicción final para diciembre
    resultado_final = diciembre[[ALMACEN, PROD_CONCAT, 'Prediccion Diciembre']].drop_duplicates()

    return resultado_final



In [107]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd

def predecir_ventas_diciembre_rf(df_ventas):
    """
    Predice las ventas de diciembre utilizando Random Forest.

    :param df_ventas: DataFrame con columnas ['Almacen', 'ProdConcat', 'Cantidad', 'Mes'].
    :return: DataFrame con predicciones de ventas para diciembre.
    """
    # Validar que el DataFrame tenga las columnas necesarias
    columnas_requeridas = [ALMACEN, PROD_CONCAT, CANTIDAD, MES]
    for columna in columnas_requeridas:
        if columna not in df_ventas.columns:
            raise ValueError(f"Falta la columna {columna} en el DataFrame.")

    # Crear una copia para no modificar el original
    df = df_ventas.copy()

    # Codificar 'Almacen' y 'ProdConcat' como números (Label Encoding)
    df[ALMACEN] = df[ALMACEN].astype('category').cat.codes
    df[PROD_CONCAT] = df[PROD_CONCAT].astype('category').cat.codes

    # Definir características (X) y la variable objetivo (y)
    X = df[[ALMACEN, PROD_CONCAT, MES]]
    y = df[CANTIDAD]

    # Dividir los datos en entrenamiento (90%) y prueba (10%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # Crear y entrenar el modelo Random Forest
    modelo_rf = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_rf.fit(X_train, y_train)

    # Hacer predicciones sobre el conjunto de prueba
    y_pred = modelo_rf.predict(X_test)

    # Calcular el error medio absoluto
    error_rf = mean_absolute_error(y_test, y_pred)
    print(f"Error Medio Absoluto (MAE) con Random Forest: {error_rf:.2f} unidades.")

    # Predecir ventas para diciembre (Mes = 12)
    diciembre = X.copy()
    diciembre[MES] = 12

    # Hacer predicción para diciembre
    diciembre[PREDICCION] = modelo_rf.predict(diciembre)

    # Descodificar 'Almacen' y 'ProdConcat' para devolver los valores originales
    diciembre[ALMACEN] = pd.Categorical.from_codes(diciembre[ALMACEN], df_ventas[ALMACEN].astype('category').cat.categories)
    diciembre[PROD_CONCAT] = pd.Categorical.from_codes(diciembre[PROD_CONCAT], df_ventas[PROD_CONCAT].astype('category').cat.categories)

    # Filtrar y mostrar predicción final para diciembre
    resultado_final = diciembre[[ALMACEN, PROD_CONCAT, 'Prediccion Diciembre']].drop_duplicates()

    return resultado_final


In [108]:
def prediccion_regresion_lineal(df_ventas_enero_nov, df_ventas_dic):
    # Llamar la función para predecir ventas de diciembre
    prediccion_diciembre_rl = predecir_ventas_diciembre_rl(df_ventas_enero_nov)

    df_sum_pred = prediccion_diciembre_rl.copy(deep=True)
    df_sum_pred = filtrar_columnas_dataframe(df_sum_pred, [PROD_CONCAT, PREDICCION])
    df_sum_pred = df_sum_pred.groupby([PROD_CONCAT]).agg({
        PREDICCION: 'sum'
    }).reset_index()

    # Definir el mapeo de columnas para renombrar
    mapeo_columnas_sum_pred = {
        PREDICCION: "Prediccion Global"
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_sum_pred.rename(columns=mapeo_columnas_sum_pred, inplace=True)

    # Realizar el pivote
    df_pivot_rl = prediccion_diciembre_rl.pivot(
        index=PROD_CONCAT,
        columns=ALMACEN,
        values=PREDICCION
    ).reset_index()

    df_pivot_rl = pd.merge(df_pivot_rl, df_sum_pred, on=[PROD_CONCAT], how="inner")

    df_sum_ventas = df_ventas_dic.copy(deep=True)
    df_sum_ventas = filtrar_columnas_dataframe(df_sum_ventas, [PROD_CONCAT, CANTIDAD])
    df_sum_ventas = df_sum_ventas.groupby([PROD_CONCAT]).agg({
        CANTIDAD: 'sum'
    }).reset_index()

        # Definir el mapeo de columnas para renombrar
    mapeo_columnas_sum_ventas = {
        CANTIDAD: "Ventas Globales"
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_sum_ventas.rename(columns=mapeo_columnas_sum_ventas, inplace=True)

    df_pivot_dic = df_ventas_dic.pivot(
        index=PROD_CONCAT,
        columns=ALMACEN,
        values=CANTIDAD
    ).reset_index()

    df_pivot_dic = pd.merge(df_pivot_dic, df_sum_ventas, on=[PROD_CONCAT], how="inner")

    # Realizar el merge entre predicciones y ventas de diciembre (REGRESION LINEAL)
    df_prediccion_v_real_dic_rl = pd.merge(df_pivot_rl, df_pivot_dic, 
                            on=[PROD_CONCAT], 
                            how="outer",  # Mantiene todos los registros (outer), inner para intersección
                            suffixes=('_pred', '_real'))  # Sufijos para diferenciar columnas
    
    return df_prediccion_v_real_dic_rl

In [109]:

def prediccion_random_forest(df_ventas_enero_nov, df_ventas_dic):
    # Llamar la función para predecir ventas de diciembre
    prediccion_diciembre_rf = predecir_ventas_diciembre_rf(df_ventas_enero_nov)

    
    df_sum_pred = prediccion_diciembre_rf.copy(deep=True)
    df_sum_pred = filtrar_columnas_dataframe(df_sum_pred, [PROD_CONCAT, PREDICCION])
    df_sum_pred = df_sum_pred.groupby([PROD_CONCAT]).agg({
        PREDICCION: 'sum'
    }).reset_index()

    # Definir el mapeo de columnas para renombrar
    mapeo_columnas_sum_pred = {
        PREDICCION: "Prediccion Global"
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_sum_pred.rename(columns=mapeo_columnas_sum_pred, inplace=True)

    # Realizar el pivote
    df_pivot_rf = prediccion_diciembre_rf.pivot(
        index=PROD_CONCAT,
        columns=ALMACEN,
        values=PREDICCION
    ).reset_index()

    df_pivot_rf = pd.merge(df_pivot_rf, df_sum_pred, on=[PROD_CONCAT], how="inner")
    
    df_sum_ventas = df_ventas_dic.copy(deep=True)
    df_sum_ventas = filtrar_columnas_dataframe(df_sum_ventas, [PROD_CONCAT, CANTIDAD])
    df_sum_ventas = df_sum_ventas.groupby([PROD_CONCAT]).agg({
        CANTIDAD: 'sum'
    }).reset_index()

        # Definir el mapeo de columnas para renombrar
    mapeo_columnas_sum_ventas = {
        CANTIDAD: "Ventas Globales"
    }
    # Renombrar las columnas del DataFrame usando el mapeo
    df_sum_ventas.rename(columns=mapeo_columnas_sum_ventas, inplace=True)

    df_pivot_dic = df_ventas_dic.pivot(
        index=PROD_CONCAT,
        columns=ALMACEN,
        values=CANTIDAD
    ).reset_index()

    df_pivot_dic = pd.merge(df_pivot_dic, df_sum_ventas, on=[PROD_CONCAT], how="inner")

    # Realizar el merge entre predicciones y ventas de diciembre (REGRESION LINEAL)
    df_prediccion_v_real_dic_rf = pd.merge(df_pivot_rf, df_pivot_dic, 
                            on=[PROD_CONCAT], 
                            how="outer",  # Mantiene todos los registros (outer), inner para intersección
                            suffixes=('_pred', '_real'))  # Sufijos para diferenciar columnas
    
    return df_prediccion_v_real_dic_rf

In [110]:
df_ventas = obtener_dataframe_ventas()

df_ventas_agrupadas_by_prodConcat = agrupa_ventas_by_prodconcat(df_ventas.copy(deep=True))

df_ventas_enero_nov = filtrar_ventas_por_mes(df_ventas=df_ventas_agrupadas_by_prodConcat.copy(deep=True), lista_meses=[1,2,3,4,5,6,7,8,9,10,11])
df_ventas_dic = filtrar_ventas_por_mes(df_ventas=df_ventas_agrupadas_by_prodConcat.copy(deep=True), lista_meses=[12])


df_prediccion_v_real_dic_rl = prediccion_regresion_lineal(df_ventas_enero_nov=df_ventas_enero_nov.copy(deep=True), df_ventas_dic=df_ventas_dic.copy(deep=True))
generar_excel_by_dataframe(df_prediccion_v_real_dic_rl, "PREDICCION-DICIEMBRE-RL")

df_prediccion_v_real_dic_rf = prediccion_random_forest(df_ventas_enero_nov=df_ventas_enero_nov.copy(deep=True), df_ventas_dic=df_ventas_dic.copy(deep=True))
generar_excel_by_dataframe(df_prediccion_v_real_dic_rf, "PREDICCION-DICIEMBRE-RF")

Ceros reemplazados por NaN en las columnas: ['Cantidad', 'PrecioVenta']
Ceros reemplazados por NaN en las columnas: ['Cantidad', 'Precio Venta']
Ceros reemplazados por NaN en las columnas: ['Cantidad', 'Precio Venta']
Error Medio Absoluto (MAE): 5.13 unidades.


C:\Users\gibra\AppData\Local\Temp\ipykernel_33668\1689136245.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_sum_pred = df_sum_pred.groupby([PROD_CONCAT]).agg({


Archivo Excel generado exitosamente: PREDICCION-DICIEMBRE-RL_2024-12-24T03-20-51.xlsx
Error Medio Absoluto (MAE) con Random Forest: 1.53 unidades.


C:\Users\gibra\AppData\Local\Temp\ipykernel_33668\3914037794.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_sum_pred = df_sum_pred.groupby([PROD_CONCAT]).agg({


Archivo Excel generado exitosamente: PREDICCION-DICIEMBRE-RF_2024-12-24T03-21-01.xlsx


'PREDICCION-DICIEMBRE-RF_2024-12-24T03-21-01.xlsx'